<h1>Supervised Learning on the labelled data</h1>
<h3>Here, we will use the supervised classification technique for the binary classification of the image patches extracted using RST as CMB or Non-CMB. The classifier used here will be Support Vector Machine(SVM) with Radial Basis Function(RBF) kernel.</h3>

In [2]:
import os
import numpy as np
import pandas as pd
from pfca import init_path
from pfca.visuals.img2D import imgplot
from pfca.core.preprocessing import *
from pfca.core.processing import *
from pfca import file_read as f
import ants
import skimage.feature as feature
from time import time
from pfca.exp.results import roi_snipping
from pfca.exp.dataset import *
from pfca import init_path

Current Working Directory: /Users/shwetankpanwar/Documents/BTPData/python_scripts/cmb_segmentation
The file structure creation failed. Structure already exists!


In [3]:
raw_dir, nifti_dir = init_path()
cur_path = os.getcwd()

<p>After the initialization, we will import the data stored in the pickle files and merge them in order to increase our sample size</p>

In [6]:
def import_dataset(dataset):
    import os
    import pandas as pd
    cur_path = os.getcwd()
    ds = pd.read_pickle(cur_path + '/datasets/learning/' + dataset)
    return ds

In [9]:
pdataset = import_dataset('rst_output_all_subjects_2_2_0.1')          #Data from positive RST
ndataset = import_dataset('rst_output_all_subjects_negative_2_2_0.1') #Data from negative RST

In [10]:
labelled_data = pdataset.append(ndataset)

In [15]:
print("Total no of labelled datapoints: " + str(len(labelled_data)))
print("Total no of target datapoints: " + str(len(labelled_data.loc[
    labelled_data['label'] == 1
])))
print("Total no of non-target datapoints: " + str(len(labelled_data.loc[
    labelled_data['label'] == 0
])))

Total no of labelled datapoints: 2599
Total no of target datapoints: 123
Total no of non-target datapoints: 2476


<h3>First, we will try to evaluate the performance on the original non-augmented dataset and see how our classifier performs</h3>
<p>The results will be used to see if the data augmentation improves performance or not.</p>
<p>The evaluation will be done based on k-fold cross validation</p>

In [17]:
#Function to perform k-fold splitting in the dataset
def k_fold_split(dataset, k, balanced = True):
        import pandas as pd
        import numpy as np
        k = dataset
        targets = k.loc[k['label'] == 1]         #Target data
        non_targets = k.loc[k['label'] == 0]     #Non target data

        n_targets = len(targets)
        n_non_t = len(non_targets)

        n_comm = min(n_targets, n_non_t)
        list_target = targets.index.values
        list_non_targets = non_targets.index.values

        if balanced == True:  # It will produce a balanced class with equal no of targets and non-targets
            # Decategorizing data in case of appending
            k['session'] = np.nan

            num = int(ratio * n_comm)  # no of target points to select

            trainees = np.random.choice(list_target, size=(num,), replace=False)
            trainees2 = np.random.choice(list_non_targets, size=(num,), replace=False)
            all_train = np.append(trainees, trainees2)
            k.loc[all_train, ['session']] = 'train'  # Labelling the datapoints as training points

            num_test = n_comm - num  # No of test target points
            target_left = k.loc[(k['label'] == 1) & (k['session'] != 'train')]
            non_targets_left = k.loc[(k['label'] == 0) & (k['session'] != 'train')]
            list_target_left = target_left.index.values
            list_non_targets_left = non_targets_left.index.values
            tests1 = np.random.choice(list_target_left, size=(num_test,), replace=False)
            tests2 = np.random.choice(list_non_targets_left, size=(num_test,), replace=False)
            all_test = np.append(tests1, tests2)
            k.loc[all_test, ['session']] = 'test'  # Labelling the datapoints as training points
            return    